In [11]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gensim.downloader as api

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [12]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


In [18]:
# Path Dataset
TRAIN_PATH = "C:/Users/USER/Documents/Data/train.csv"
TEST_PATH = "C:/Users/USER/Documents/Data/test.csv"

In [19]:
# Load Dataset
train_df = pd.read_csv(TRAIN_PATH, header=None, names=["class", "title", "description"])
test_df = pd.read_csv(TEST_PATH, header=None, names=["class", "title", "description"])


In [20]:
# Gabungkan title dan description
train_df["text"] = train_df["title"].fillna("") + ". " + train_df["description"].fillna("")
test_df["text"] = test_df["title"].fillna("") + ". " + test_df["description"].fillna("")

In [21]:
# Encode label
label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["class"])
test_df["label"] = label_encoder.transform(test_df["class"])

In [22]:
# Load GloVe pre-trained embeddings
print("Loading GloVe pre-trained embeddings...")
glove = api.load("glove-wiki-gigaword-300")  # GloVe 300-dimensional
EMBEDDING_DIM = 300

Loading GloVe pre-trained embeddings...


NameError: name 'api' is not defined

In [8]:
# Bangun embedding matrix
VOCAB_SIZE = len(glove.key_to_index) + 2  # Tambahkan padding dan unknown token
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
embedding_matrix = [np.zeros(EMBEDDING_DIM), np.random.uniform(-0.25, 0.25, EMBEDDING_DIM)]

In [9]:
# Isi matrix embedding dari GloVe
for idx, word in enumerate(glove.key_to_index):
    word_to_idx[word] = idx + 2
    embedding_matrix.append(glove[word])

embedding_matrix = np.array(embedding_matrix)
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float)
print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: torch.Size([400002, 300])


In [10]:
from transformers import BertTokenizer, BertModel

In [11]:
# Load BERT tokenizer dan model
print("Loading BERT pre-trained model...")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Loading BERT pre-trained model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
# Dataset
class CombinedDataset(Dataset):
    def __init__(self, texts, labels, word_to_idx, max_len, bert_tokenizer):
        self.texts = texts
        self.labels = labels
        self.word_to_idx = word_to_idx
        self.max_len = max_len
        self.bert_tokenizer = bert_tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenisasi BERT
        bert_encoding = self.bert_tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        input_ids_bert = bert_encoding["input_ids"].squeeze(0)
        attention_mask = bert_encoding["attention_mask"].squeeze(0)

        # Tokenisasi GloVe
        tokens = text.lower().split()
        token_ids_glove = [self.word_to_idx.get(token, 1) for token in tokens]  # 1 = "<UNK>"
        if len(token_ids_glove) < self.max_len:
            token_ids_glove += [0] * (self.max_len - len(token_ids_glove))  # 0 = "<PAD>"
        else:
            token_ids_glove = token_ids_glove[:self.max_len]

        return {
            "input_ids_bert": input_ids_bert,
            "attention_mask": attention_mask,
            "input_ids_glove": torch.tensor(token_ids_glove, dtype=torch.long),
            "label": torch.tensor(label, dtype=torch.long)
        }


In [13]:
# Dataset dan DataLoader
MAX_LEN = 100
train_dataset = CombinedDataset(train_df["text"].values, train_df["label"].values, word_to_idx, MAX_LEN, bert_tokenizer)
test_dataset = CombinedDataset(test_df["text"].values, test_df["label"].values, word_to_idx, MAX_LEN, bert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [14]:
# Model
class BERTGloVeClassifier(nn.Module):
    def __init__(self, bert_model, embedding_matrix, hidden_size, n_classes, dropout=0.5):
        super(BERTGloVeClassifier, self).__init__()
        self.bert = bert_model
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)

        # Fully connected layers
        self.fc_glove = nn.Linear(EMBEDDING_DIM, hidden_size)
        self.fc_bert = nn.Linear(bert_model.config.hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size * 2, n_classes)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids_bert, attention_mask, input_ids_glove):
        # BERT forward pass
        bert_output = self.bert(input_ids=input_ids_bert, attention_mask=attention_mask)
        cls_output = bert_output.last_hidden_state[:, 0, :]  # [CLS] token output

        # GloVe forward pass
        glove_embeddings = self.embedding(input_ids_glove)
        glove_output = glove_embeddings.mean(dim=1)  # Mean pooling

        # Fully connected layers
        bert_features = self.dropout(torch.relu(self.fc_bert(cls_output)))
        glove_features = self.dropout(torch.relu(self.fc_glove(glove_output)))

        combined_features = torch.cat((bert_features, glove_features), dim=1)
        return self.fc_out(combined_features)


In [15]:
# Inisialisasi Model
model = BERTGloVeClassifier(
    bert_model=bert_model,
    embedding_matrix=embedding_matrix,
    hidden_size=128,
    n_classes=len(label_encoder.classes_)
).to(DEVICE)

In [16]:
# Optimizer dan Loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


In [17]:
# Training Function
def train_epoch(model, data_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids_bert = batch["input_ids_bert"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        input_ids_glove = batch["input_ids_glove"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids_bert, attention_mask, input_ids_glove)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)


In [18]:
# Evaluasi
def evaluate(model, data_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids_bert = batch["input_ids_bert"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            input_ids_glove = batch["input_ids_glove"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            outputs = model(input_ids_bert, attention_mask, input_ids_glove)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return accuracy_score(true_labels, predictions)


In [ ]:
# Train Model
EPOCHS = 3
for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Accuracy: {acc:.4f}")

In [ ]:
torch.save(model.state_dict(), "bert_with_glove.pth")
print("Model saved successfully!")